# GLM Demo: multivariate noise stimulus

This is a demo notebook for the GLM estimation algorithm. We simulate data using the GLM, and then try to estimate parameters with the GLM. This notebook demos the use of a multivariate stimulus like a spectrogram, although the stimulus used here is more like 1D visual white noise.

In [1]:
from __future__ import print_function, division
import os
import sys
import numpy as np

import mat_neuron._model as mat
from dstrf import io, strf, mle, simulate, stimulus, filters, models, spikes, performance

# plotting packages
%reload_ext yamlmagic
%matplotlib inline
import matplotlib.pyplot as plt # plotting functions
import seaborn as sns           # data visualization package
sns.set_style("whitegrid")

cfg = {}

Set up parameters using YAML and Munch

In [2]:
%%yaml cfg
model:
  dt: 0.5
  ataus: [10.0, 200.0]
  t_refract: 2.0
  filter:
    rank: 2
    len: 30
    ncos: 8
data:
  stimulus:
    duration: 400000
  model: "multivariate_glm"
  filter:
    fn: "gabor"
    nfreq: 14
    ntau: 30
    f_max: 8000
    f_peak: 4000
    t_peak: 12
    ampl: -1
    t_sigma: 6
    f_sigma: 2400
    theta: 0
    lmbda: 19
    psi: 1.5
  adaptation: [7.0, 100.0, 2.0]
  trial_noise:
    sd: 2.0
  random_seed: 1
  dt: 10.0
  trials: 5

<IPython.core.display.Javascript object>

In [3]:
from munch import munchify
cf = munchify(cfg)

In [ ]:
k1, _, _ = filters.gabor(**cf.data.filter)
plt.imshow(k1, cmap='jet', aspect='auto');

The simulation code has been factored out to the `simulate` module.

In [ ]:
import imp
imp.reload(io)
assim_data = stimulus.randn(cf)
assim_data = io.merge_data(simulate.multivariate_glm(cf, assim_data))
t_stim = np.arange(0, cf.data.stimulus.duration, cf.data.dt)

fig, axes = plt.subplots(nrows=3, ncols=1, sharex=True, figsize=(6, 4))
axes[0].imshow(assim_data["stim"], extent=(0, cf.data.stimulus.duration, 0, cf.data.filter.ntau), 
               origin='lower', aspect='auto')
axes[1].plot(t_stim, assim_data["V"])
for i, spk in enumerate(assim_data["spike_t"]):
    axes[2].vlines(spk * cf.model.dt, i, i + 0.5)

axes[0].set_xlim(1000, 2000);

## Estimate parameters

First we'll estimate the full RF (using the raised cosine basis set to compress time)

In [ ]:
# initial guess of parameters using ML
kcosbas = strf.cosbasis(cf.model.filter.len, cf.model.filter.ncos)
try:
    mlest = mle.mat(assim_data["stim"], kcosbas, assim_data["spike_v"], assim_data["spike_h"],
                        assim_data["stim_dt"], assim_data["spike_dt"])
except TypeError:
    mlest = mle.mat(assim_data["stim"], kcosbas, assim_data["spike_v"], assim_data["spike_h"],
                        assim_data["stim_dt"], assim_data["spike_dt"])
%time w0 = mlest.estimate()

In [ ]:
print("True rate and adaptation parameters:", cf.data.adaptation)
print("MLE rate and adaptation parameters:", w0[:3])
fig, axes = plt.subplots(nrows=2, ncols=2, sharex=True, sharey=True, figsize=(6, 6))

k1c = strf.to_basis(k1, kcosbas)
rf_sta = strf.as_matrix(mlest.sta(), kcosbas)
rf_mle = strf.as_matrix(w0[3:], kcosbas)
axes[0, 0].imshow(k1, cmap='jet', aspect='auto')
axes[0, 0].set_title("True RF")
axes[0, 1].imshow(strf.from_basis(k1c, kcosbas), cmap='jet', aspect='auto')
axes[0, 1].set_title("RF from cosine basis")
axes[1, 0].imshow(rf_sta, cmap='jet', aspect='auto')
axes[1, 0].set_title("STA")
axes[1, 1].imshow(rf_mle, cmap='jet', aspect='auto')
axes[1, 1].set_title("MLE")

We can also reduce parameter counts by factorizing the STRF (as in Thorson et al). This doesn't make much of a difference with white noise stimuli but it can really help when there are strong correlations.

In [ ]:
krank = 1
mlest = mle.matfact(assim_data["stim"], kcosbas, krank, assim_data["spike_v"], assim_data["spike_h"],
                        assim_data["stim_dt"], assim_data["spike_dt"])
%time w0 = mlest.estimate()

In [ ]:
print("True rate and adaptation parameters:", cf.data.adaptation)
print("MLE rate and adaptation parameters:", w0[:3])
fig, axes = plt.subplots(nrows=2, ncols=2, sharex=True, sharey=True, figsize=(6, 6))

k1c = strf.to_basis(k1, kcosbas)
rf_sta = strf.as_matrix(mlest.sta(), kcosbas)
rf_mle = strf.from_basis(strf.defactorize(w0[3:], cf.data.filter.nfreq, krank), kcosbas)
axes[0, 0].imshow(k1, cmap='jet', aspect='auto')
axes[0, 0].set_title("True RF")
axes[0, 1].imshow(strf.from_basis(k1c, kcosbas), cmap='jet', aspect='auto')
axes[0, 1].set_title("RF from cosine basis")
axes[1, 0].imshow(rf_sta, cmap='jet', aspect='auto')
axes[1, 0].set_title("STA")
axes[1, 1].imshow(rf_mle, cmap='jet', aspect='auto')
axes[1, 1].set_title("MLE (rank-{})".format(krank));

## Predict responses

In [ ]:
test_stim = stimulus.randn(cf, random_seed=1000)
test_data = io.merge_data(simulate.multivariate_glm(cf, test_stim, random_seed=1000, trials=10))
mltest = mle.matfact(test_data["stim"], kcosbas, krank, test_data["spike_v"], test_data["spike_h"],
                     test_data["stim_dt"], test_data["spike_dt"])

In [ ]:
fig, axes = plt.subplots(nrows=4, ncols=1, sharex=True)
axes[0].imshow(test_data["stim"], extent=(0, cf.data.stimulus.duration, 0, cf.data.filter.ntau), 
               origin='lower', aspect='auto')

t_stim = np.linspace(0, test_data["duration"], test_data["stim"].shape[1])
t_spike = np.linspace(0, test_data["duration"], test_data["spike_v"].shape[0])
V = strf.convolve(test_data["stim"], k1)
Vpred = mltest.V(w0)
axes[1].plot(t_stim, V, t_stim, Vpred)

n_trials = test_data["ntrials"]
for i, spk in enumerate(test_data["spike_t"]):
    axes[2].vlines(spk * cf.model.dt, i - 0.4 + n_trials, i + 0.4 + n_trials)
pred = np.zeros_like(test_data["spike_v"])
for j in range(n_trials):
    pred[:, j] = models.predict_spikes_glm(Vpred, w0[:3], cf)
    spk_t = pred[:, j].nonzero()[0]
    axes[2].vlines(spk_t * cf.model.dt, j - 0.4, j + 0.4, color='r')

upsample = int(cf.data.dt / cf.model.dt)   
pred_psth = spikes.psth(pred, upsample, 1)
test_psth = spikes.psth(test_data["spike_v"], upsample, 1)
axes[3].plot(t_stim, test_psth, t_stim, pred_psth)
axes[3].set_xlim(0, 2000);

eo = performance.corrcoef(test_data["spike_v"][::2], test_data["spike_v"][1::2], upsample, 1)
print("EO cc: %3.3f" % eo)
print("pred cc: %3.3f" % np.corrcoef(test_psth, pred_psth)[0, 1])
print("spike count: data = {}, pred = {}".format(test_data["spike_v"].sum(), pred.sum()))